In [1]:
from omegaconf import OmegaConf
import torch

config_file = "./custom_configs/7B_lora.yaml"

In [2]:
cfg = OmegaConf.load(config_file)

In [3]:
from torchtune.config._instantiate import instantiate

/usr/local/share/miniconda/envs/torchtune/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
scheduler_cfg = cfg.profile.schedule
scheduler_cfg.repeat = 0

default_scheduler = {
    "_component_": "torch.profiler.schedule",
    "wait": 1,
    "warmup": 1,
    "active": 1,
    "repeat": 0,
}

In [5]:
OmegaConf.select(cfg.profile, "schedule", default=None, throw_on_missing=False)

{'_component_': 'torch.profiler.schedule', 'wait': 1, 'warmup': 1, 'active': 1, 'repeat': 0}

In [54]:
test_schedule = instantiate(scheduler_cfg)

In [55]:
ref_schedule = torch.profiler.schedule(wait=1, warmup=1, active=1)

In [56]:
test_actions = [test_schedule(i) for i in range(5)]
ref_actions = [ref_schedule(i) for i in range(5)]
assert test_actions == ref_actions

In [90]:
cfg = OmegaConf.load(config_file)
cfg.profile.profiler

{'_component_': 'torch.profiler.profile', 'record_shapes': True, 'profile_memory': True, 'with_stack': True, 'with_flops': True}

In [91]:
cfg.profile.CUDA
OmegaConf.select(cfg.profile, "CPU", default=True, throw_on_missing=False)

True

In [77]:
p = instantiate(
    cfg.profile.profiler,
    activities=[
        torch.profiler.ProfilerActivity.CPU if cfg.profile.CPU else None,
        torch.profiler.ProfilerActivity.CUDA if cfg.profile.CUDA else None,
    ],
)

In [96]:
p.schedule = test_schedule

In [99]:
p.with_stack

True

In [95]:
[p.schedule(i) for i in range(5)]

[<ProfilerAction.NONE: 0>,
 <ProfilerAction.WARMUP: 1>,
 <ProfilerAction.RECORD_AND_SAVE: 3>,
 <ProfilerAction.NONE: 0>,
 <ProfilerAction.WARMUP: 1>]

In [81]:
assert all(
    a in p.activities
    for a in [torch.profiler.ProfilerActivity.CPU, torch.profiler.ProfilerActivity.CUDA]
)

In [82]:
ref_prof = torch.profiler.profile(
    activities=[
        torch.profiler.ProfilerActivity.CPU,
        torch.profiler.ProfilerActivity.CUDA,
    ],
    record_shapes=True,
    profile_memory=True,
    with_stack=True,
    with_flops=True,
)

In [84]:
assert p.activities == ref_prof.activities